# Diário de Bordo

In [2]:
import pandas as pd

# Lendo o arquivo csv com o delimitador ';'
info_transportes = pd.read_csv('info_transportes.csv', delimiter=';')

info_transportes.head()

,DATA_INICIO,DATA_FIM,CATEGORIA,LOCAL_INICIO,LOCAL_FIM,DISTANCIA,PROPOSITO
0,01-01-2016 21:11,01-01-2016 21:17,Negocio,Fort Pierce,Fort Pierce,51,Alimentação
1,01-02-2016 01:25,01-02-2016 01:37,Negocio,Fort Pierce,Fort Pierce,5,NaN
2,01-02-2016 20:25,01-02-2016 20:38,Negocio,Fort Pierce,Fort Pierce,48,Entregas
3,01-05-2016 17:31,01-05-2016 17:45,Negocio,Fort Pierce,Fort Pierce,47,Reunião
4,01-06-2016 14:42,01-06-2016 15:49,Negocio,Fort Pierce,West Palm Beach,637,Visita ao cliente


In [3]:
#Verificando mais informações sobre o df
info_transportes.info()
info_transportes.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1153 entries, 0 to 1152
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   DATA_INICIO   1153 non-null   object
 1   DATA_FIM      1153 non-null   object
 2   CATEGORIA     1153 non-null   object
 3   LOCAL_INICIO  1153 non-null   object
 4   LOCAL_FIM     1153 non-null   object
 5   DISTANCIA     1153 non-null   int64 
 6   PROPOSITO     651 non-null    object
dtypes: int64(1), object(6)
memory usage: 63.2+ KB


DATA_INICIO       0
DATA_FIM          0
CATEGORIA         0
LOCAL_INICIO      0
LOCAL_FIM         0
DISTANCIA         0
PROPOSITO       502
dtype: int64

In [4]:
#Transformando as colunas DATA_INICIO e DATA_FIM em datetime
info_transportes['DATA_INICIO'] = pd.to_datetime(info_transportes['DATA_INICIO'], format='%m-%d-%Y %H:%M')
info_transportes['DATA_FIM'] = pd.to_datetime(info_transportes['DATA_FIM'], format='%m-%d-%Y %H:%M')

#Transformando as colunas DATA_INICIO e DATA_FIM para o formato yyyy-MM-dd
info_transportes['DATA_INICIO'] = info_transportes['DATA_INICIO'].dt.strftime('%Y-%m-%d')
info_transportes['DATA_FIM'] = info_transportes['DATA_FIM'].dt.strftime('%Y-%m-%d')

info_transportes['DATA_INICIO'] = pd.to_datetime(info_transportes['DATA_INICIO'])
info_transportes['DATA_FIM'] = pd.to_datetime(info_transportes['DATA_FIM'])


## Criando o dataframe 'info_corridas_do_dia'

In [5]:
#Criando o novo dataframe com a quantidade de corridas por dia
info_corridas_do_dia = info_transportes.groupby('DATA_INICIO').size().reset_index(name='QT_CORRIDAS').astype({'QT_CORRIDAS': 'int32'})

info_corridas_do_dia = info_corridas_do_dia.rename(columns={'DATA_INICIO': 'DT_REFE'})

info_corridas_do_dia.head()

,DT_REFE,QT_CORRIDAS
0,2016-01-01,1
1,2016-01-02,2
2,2016-01-05,1
3,2016-01-06,3
4,2016-01-07,1


In [6]:
#Corridas da categoria "Negocio"

df = info_transportes[info_transportes['CATEGORIA'] == 'Negocio'].groupby('DATA_INICIO').size().reset_index(name='QT_CORR_NEG').astype({'QT_CORR_NEG': 'int32'})
df = df.rename(columns={'DATA_INICIO': 'DT_REFE'})

info_corridas_do_dia = pd.merge(info_corridas_do_dia, df, on='DT_REFE', how='left')

#Preenchendo os valores nulos com 0
info_corridas_do_dia['QT_CORR_NEG'] = info_corridas_do_dia['QT_CORR_NEG'].fillna(0).astype(int)

In [7]:
#Corridas da categoria "Pessoal"

df = info_transportes[info_transportes['CATEGORIA'] == 'Pessoal'].groupby('DATA_INICIO').size().reset_index(name='QT_CORR_PESS').astype({'QT_CORR_PESS': 'int32'})
df = df.rename(columns={'DATA_INICIO': 'DT_REFE'}) 

info_corridas_do_dia = pd.merge(info_corridas_do_dia, df, on='DT_REFE', how='left')

#Preenchendo os valores nulos com 0
info_corridas_do_dia['QT_CORR_PESS'] = info_corridas_do_dia['QT_CORR_PESS'].fillna(0).astype(int)

In [8]:
#Maior distancia percorrida por uma corrida

df = info_transportes.groupby('DATA_INICIO')['DISTANCIA'].max().reset_index(name='VL_MAX_DIST').astype({'VL_MAX_DIST': 'float32'})
df = df.rename(columns={'DATA_INICIO': 'DT_REFE'})

info_corridas_do_dia = pd.merge(info_corridas_do_dia, df, on='DT_REFE', how='left')

In [9]:
#Menor distancia percorrida por uma corrida

df = info_transportes.groupby('DATA_INICIO')['DISTANCIA'].min().reset_index(name='VL_MIN_DIST').astype({'VL_MIN_DIST': 'float32'})
df = df.rename(columns={'DATA_INICIO': 'DT_REFE'})

info_corridas_do_dia = pd.merge(info_corridas_do_dia, df, on='DT_REFE', how='left')

In [10]:
#Média das distancias percorridas

df = info_transportes.groupby('DATA_INICIO')['DISTANCIA'].mean().reset_index(name='VL_AVG_DIST').astype({'VL_AVG_DIST': 'float32'})
df = df.rename(columns={'DATA_INICIO': 'DT_REFE'})

info_corridas_do_dia = pd.merge(info_corridas_do_dia, df, on='DT_REFE', how='left')

In [11]:
#Quantidade de corridas com o propósito "Reunião"

df = info_transportes[info_transportes['PROPOSITO'] == 'Reunião'].groupby('DATA_INICIO').size().reset_index(name='QT_CORR_REUNI').astype({'QT_CORR_REUNI': 'int32'})
df = df.rename(columns={'DATA_INICIO': 'DT_REFE'})

info_corridas_do_dia = pd.merge(info_corridas_do_dia, df, on='DT_REFE', how='left')

#Preenchendo os valores nulos com 0
info_corridas_do_dia['QT_CORR_REUNI'] = info_corridas_do_dia['QT_CORR_REUNI'].fillna(0).astype(int)

In [12]:
#Quantidade de corridas com o propósito declarado e diferente de "Reunião"

#Definindo as duas regras antes do agrupamento com notna() e diferente de "Reunião"
df = info_transportes[info_transportes['PROPOSITO'].notna() & (info_transportes['PROPOSITO'] != 'Reunião')].groupby('DATA_INICIO').size().reset_index(name='QT_CORR_NAO_REUNI').astype({'QT_CORR_NAO_REUNI': 'int32'})
df = df.rename(columns={'DATA_INICIO': 'DT_REFE'})

info_corridas_do_dia = pd.merge(info_corridas_do_dia, df, on='DT_REFE', how='left')

#Preenchendo os valores nulos com 0
info_corridas_do_dia['QT_CORR_NAO_REUNI'] = info_corridas_do_dia['QT_CORR_NAO_REUNI'].fillna(0).astype(int)

In [13]:
#Corrigindo as casas decimais
info_corridas_do_dia['VL_AVG_DIST'] = info_corridas_do_dia['VL_AVG_DIST'].apply(lambda x: round(x, 1))

#Tentei com 'info_transportes['VL_AVG_DIST'] = info_transportes['VL_AVG_DIST'].round(1)' Mas não deu certo

In [14]:
#Salvando o DataFrame em um arquivo CSV
info_corridas_do_dia.to_csv('info_corridas_do_dia.csv', index=False)

In [15]:
info_corridas_do_dia.head()

,DT_REFE,QT_CORRIDAS,QT_CORR_NEG,QT_CORR_PESS,VL_MAX_DIST,VL_MIN_DIST,VL_AVG_DIST,QT_CORR_REUNI,QT_CORR_NAO_REUNI
0,2016-01-01,1,1,0,51.0,51.0,51.0,0,1
1,2016-01-02,2,2,0,48.0,5.0,26.5,0,1
2,2016-01-05,1,1,0,47.0,47.0,47.0,1,0
3,2016-01-06,3,3,0,637.0,43.0,250.3,1,2
4,2016-01-07,1,1,0,8.0,8.0,8.0,1,0


In [16]:
df = pd.read_csv('info_corridas_do_dia.csv')